In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy as d_copy
import random

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#error_index = 0
vgg16_bn = torchvision.models.vgg16_bn(pretrained=True)#.to(device)
print(device)

cuda


In [3]:
from importnb import Notebook

In [4]:
with Notebook(lazy=True):
    import F4F_model
header = F4F_model

In [5]:
# set randomness
seed = 0

header.set_randomness(seed) # ipynb module import means run all cell in file
# load dataset
dataset_path = "/media/2/Network/Imagenet_dup/"
retrain_model_path = "/media/0/Network/0821_to_fullmodels/"
batch_size = 32 # 32~ out of memory in 3080
num_train = 128000 #640000

train_dataloader,test_dataloader = header.get_dataset(num_train,batch_size,
                  dataset_path,retrain_model_path)

===INFO===
torch ver : 1.8.0
torchvision ver : 0.2.2 
GPU model : TITAN RTX
train dataset[4000], test dataset[1563] are loaded


In [6]:
#In_layer_number = 34 # 34 conv5_1 convolution
#Out_layer_number = 36 # 36 conv5_1 relu 
error_index=0
max_epoch = 90
num_error = 64

In [7]:
log_file = "./3fc_64_acc_log_0826.txt"

In [8]:
f4f = header.F4F().to(device)
optimizer = torch.optim.SGD(f4f.parameters(),lr=1e-3,weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()


In [9]:
f4f.parameters

<bound method Module.parameters of F4F(
  (fc1): Linear(in_features=5120, out_features=5120, bias=True)
  (fc2): Linear(in_features=5120, out_features=5120, bias=True)
  (fc3): Linear(in_features=5120, out_features=4608, bias=True)
)>

In [10]:
if torch.cuda.device_count() >1 :
    print("data parallel start")
    f4f = nn.DataParallel(f4f).to(device)

In [11]:
#split_model = split_layer(vgg16_bn,0,Out_layer_number)

original_model = d_copy(vgg16_bn).to(device)
# subset of vgg16 (whole layer) with f4f
#header.hook_register(vgg16_bn,error_index,num_error)
header.hook_register(vgg16_bn)
target_model = header.Target_model(vgg16_bn).to(device)



In [12]:
with Notebook(lazy=True):
    import F4F_model
header = F4F_model

In [ ]:

#optimizer = torch.optim.SGD(param_list,lr=0.01,weight_decay=1e-4)
first_feature = []
first_label = []
offset_info = []
f = open(log_file,"w")
print("3fc layer concatenate error_index & filter_set")
f.close()

header.hook_register(vgg16_bn)
target_model = header.Target_model(vgg16_bn).to(device)
    
tmp= header.training(f4f,target_model,original_model,
                  train_dataloader,test_dataloader,batch_size,
                  log_file,retrain_model_path,
                  loss_fn,optimizer,
                  num_error,max_epoch,True)
        # tmp : first_feature,first_label,offset_info
first_feature.append(tmp[0])
first_label.append(tmp[1])
offset_info.append(tmp[2])

3fc layer concatenate error_index & filter_set
..........
[1,   100] loss: 0.251065
..........
[1,   200] loss: 0.247853
..........
[1,   300] loss: 0.245431
..........
[1,   400] loss: 0.247493
..........
[1,   500] loss: 0.248858
..........
[1,   600] loss: 0.248526
..........
[1,   700] loss: 0.249348
..........
[1,   800] loss: 0.246621
..........
[1,   900] loss: 0.245774
..........
[1,  1000] loss: 0.249021
..........
[1,  1100] loss: 0.249191
..........
[1,  1200] loss: 0.248932
..........
[1,  1300] loss: 0.246489
..........
[1,  1400] loss: 0.247000
..........
[1,  1500] loss: 0.248833
..........
[1,  1600] loss: 0.246210
..........
[1,  1700] loss: 0.247872
..........
[1,  1800] loss: 0.246139
..........
[1,  1900] loss: 0.247689
..........
[1,  2000] loss: 0.250959
..........
[1,  2100] loss: 0.246029
..........
[1,  2200] loss: 0.248425
..........
[1,  2300] loss: 0.247692
..........
[1,  2400] loss: 0.245659
..........
[1,  2500] loss: 0.249281
..........
[1,  2600] loss: 

In [ ]:
# 여기서부터는 feature 그림 보기 위한 것들입니다.
for i in range(10):
    print(first_feature[0][0][i],original_out[0][0][i].argmax())

In [ ]:

w = 10
h = 10
cols = 32
rows = 16
def feature_print(pic):
    #print("test with 'after pooling 4 feature'")
    fig = plt.figure(figsize=(64,32))
    ax = []
    for i in range(cols*rows):
        ch = pic[i,:,:]
        ax.append(fig.add_subplot(rows,cols,i+1))
        ax[-1].set_title(str(i)+"th ch (14x14)")
        plt.imshow(ch)

In [ ]:
feature_print(first_feature[0])